In [1]:
import pandas as pd
import re
import demoji
#demoji.download_codes()
import fasttext
import fasttext.util

In [2]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/constraint_Hindi_Train - Sheet1.csv"
data = pd.read_csv(file_path)
data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [3]:
# making labels for fasttext input
y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':1,'fake':1,'defamation':1}
for labels in data['Labels Set']:
    temp = list(set(['__label__' + str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y[:10]

['__label__1',
 '__label__0',
 '__label__0',
 '__label__1',
 '__label__0',
 '__label__1',
 '__label__0',
 '__label__0',
 '__label__0',
 '__label__0']

In [4]:
def hindi_tweet_cleaning(tweet):
    tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet)
    pat = re.compile(r"([',./@#(;_`:|\-)!])")
    tweet = pat.sub("", tweet)
    tweet_split_word_list = tweet.split()
    tweet_split_word_list_new = []
    #print(tweet_split_word_list)
    for word in tweet_split_word_list:
        if not word.isalpha():
            tweet_split_word_list_new.append(word)
    tweet = " ".join(tweet_split_word_list_new)
    tweet = demoji.replace(tweet,"")
    tweet = tweet.rstrip("\n")
    tweet = re.sub('\n', '', tweet)
    tweet = tweet.replace('\u200b', '')
    tweet = tweet.replace('\u200c', '')
    tweet = tweet.replace('\u200d', '')
    return tweet

In [5]:
all_tweets= []
for tweet in data['Post']:
    all_tweets.append(hindi_tweet_cleaning(tweet))

In [6]:
training_data = []
for tweet, label in zip(all_tweets,y):
    training_data.append(label+" "+tweet)

In [7]:
training_data_ = training_data[:5500]
validation_data = training_data[5500:]
# testing_data = training_data[5200:]

In [8]:
training_data_[0]

'__label__1 मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है कुछ बाबर के साले है  श्री राम '

In [9]:
f = open('train.txt', 'w')
for i in training_data_:
    f.write(i+"\n")
f.close()

f = open('valid.txt', 'w')
for i in validation_data:
    f.write(i+"\n")
f.close()

In [10]:
# model = fasttext.train_supervised(input='train.txt')
model = fasttext.train_supervised(input="train.txt", epoch = 3,wordNgrams=2,loss='ova', lr= 0.7)

In [11]:
# model.save_model("/content/drive/My Drive/devanagri_text_classification/modelepoch=50,wordNgrams=2,loss='ova', lr=0.7.bin")

In [12]:
model.test('train.txt')

(5500, 0.9992727272727273, 0.9992727272727273)

In [13]:
model.test("valid.txt")
# f1 score = 70

(228, 0.8640350877192983, 0.8640350877192983)

In [14]:
model.predict("ईरान के सर्वोच्च नेता अयातुल्लाह अली खामेनई ने यूएई मुस्लिम के साथ धोखा करने का आरोप लगाया",k=3)

(('__label__0', '__label__1'), array([0.99219412, 0.00758724]))

In [15]:
# testing

In [16]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/Constraint_Hindi_Valid - Sheet1.csv"
data = pd.read_csv(file_path)

In [17]:
# making labels for fasttext input
y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':1,'fake':1,'defamation':1}
for labels in data['Labels Set']:
    temp = list(set(['__label__' + str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y[:10]

['__label__0',
 '__label__1',
 '__label__0',
 '__label__0',
 '__label__0',
 '__label__0',
 '__label__0',
 '__label__1',
 '__label__0',
 '__label__0']

In [18]:
all_tweets= []
for tweet in data['Post']:
    all_tweets.append(hindi_tweet_cleaning(tweet))

In [19]:
testing_data = []
for tweet, label in zip(all_tweets,y):
    testing_data.append(label+" "+tweet)

In [20]:
f = open('test.txt', 'w')
for i in testing_data:
    f.write(i+'\n')
f.close()

In [21]:
model.test("test.txt")

(811, 0.8840937114673243, 0.8840937114673243)